In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import time


In [3]:
# Input and output file paths
input_file = "data/energe_res_price.csv"  # Update the path if needed
output_file = "data/energe_res_price_with_coords.csv"  # Path to save the output

# Load the dataset
df = pd.read_csv(input_file)
df.head()


,the_geom,cartodb_id,the_geom_webmercator,id,bbl,bin,address,zipcode,bldgtype,idflags,...,eui_base,eui_diff,wui_base,wui_diff,ghg_base,ghg_diff,fuel_mix,address_standard,pricepersqft,price
0,NaN,207,NaN,14441087,2.058031e+09,2084227,4415 Post Road,10471,Residence Hall/Dormitory,0,...,133.1,-0.021037,31.52,0.078997,5.42,-0.038745,"{'Fuel Oil 4': nan, 'Fuel Oil 2': nan, 'Natura...",4415 post road,NaN,NaN
1,NaN,1616,NaN,12003204,3.021738e+09,3251688,145 Taylor Street,11211,Multifamily Housing,0,...,425.7,-0.797040,51.66,0.003291,13.84,-0.752168,"{'Fuel Oil 4': nan, 'Fuel Oil 2': nan, 'Natura...",145 taylor street,NaN,NaN
2,NaN,1912,NaN,20313612,1.008160e+09,1015299,40 West 15th Street,10011,Multifamily Housing,0,...,44.1,1.045351,26.57,-0.045164,1.38,2.289855,"{'Fuel Oil 4': nan, 'Fuel Oil 2': 1124272.2, '...",40 west 15th street,NaN,NaN
3,NaN,2207,NaN,12751380,2.041960e+09,2046612,3063 Buhre Avenue,10461,Multifamily Housing,0,...,112.3,0.138023,64.33,0.017410,6.89,0.153846,"{'Fuel Oil 4': 3796584.0, 'Fuel Oil 2': nan, '...",3063 buhre avenue,NaN,NaN
4,NaN,2449,NaN,16871963,2.039470e+09,2041222,2145 Starling Avenue,10462,Multifamily Housing,0,...,199.9,-0.291646,85.07,-0.998589,13.17,-0.537585,"{'Fuel Oil 4': nan, 'Fuel Oil 2': nan, 'Natura...",2145 starling avenue,NaN,NaN


In [4]:
# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapiExercises")


In [5]:
# Function to get latitude and longitude with retries
def get_lat_lon_with_retry(address, retries=3, delay=2):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.longitude, location.latitude
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for address {address}: {e}")
            time.sleep(delay * (attempt + 1))  # Exponential backoff
    return None, None


In [6]:
# Batch size for processing
batch_size = 50
processed_df = pd.DataFrame()  # Create an empty dataframe for results

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    print(f"Processing batch {i} to {i + batch_size - 1}...")

    # Geocode each address
    lon_lat_results = batch['address'].apply(lambda addr: get_lat_lon_with_retry(addr) if pd.notnull(addr) else (None, None))
    batch['longitude'], batch['latitude'] = zip(*lon_lat_results)

    # Append the batch to the processed dataframe
    processed_df = pd.concat([processed_df, batch], ignore_index=True)

    # Save intermediate results to a file
    processed_df.to_csv(output_file, index=False)
    print(f"Saved batch {i} to {i + batch_size - 1}.")


Processing batch 0 to 49...
Attempt 1 failed for address 4415 Post Road: HTTP Error 403: Forbidden
Attempt 2 failed for address 4415 Post Road: HTTP Error 403: Forbidden
Attempt 3 failed for address 4415 Post Road: HTTP Error 403: Forbidden
Attempt 1 failed for address 145 Taylor Street: HTTP Error 403: Forbidden
Attempt 2 failed for address 145 Taylor Street: HTTP Error 403: Forbidden
Attempt 3 failed for address 145 Taylor Street: HTTP Error 403: Forbidden
Attempt 1 failed for address 40 West 15th Street: HTTP Error 403: Forbidden
Attempt 2 failed for address 40 West 15th Street: HTTP Error 403: Forbidden
Attempt 3 failed for address 40 West 15th Street: HTTP Error 403: Forbidden
Attempt 1 failed for address 3063 Buhre Avenue: HTTP Error 403: Forbidden
Attempt 2 failed for address 3063 Buhre Avenue: HTTP Error 403: Forbidden
Attempt 3 failed for address 3063 Buhre Avenue: HTTP Error 403: Forbidden
Attempt 1 failed for address 2145 Starling Avenue: HTTP Error 403: Forbidden
Attempt 2 

KeyboardInterrupt: 